# 彩色图片分类

## 加载相关库

In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers
import matplotlib.pyplot as plt

## 导入数据

In [5]:
(train_x, train_y), (test_x, test_y) = datasets.cifar10.load_data()

170500096/170498071 [==============================] - 94s 1us/step


## 归一化

In [6]:
train_x, test_x = train_x / 255.0, test_x / 255.0

print("train_data: ", train_x.shape, train_y.shape)
print("test_data: ", test_x.shape, test_y.shape)

train_data:  (50000, 32, 32, 3) (50000, 1)
test_data:  (10000, 32, 32, 3) (10000, 1)


## 建模

In [16]:
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_1 = layers.Conv2D(32, (3, 3), activation='relu')
        self.conv_2 = layers.Conv2D(64, (3, 3), activation='relu')
        self.conv_3 = layers.Conv2D(64, (3, 3), activation='relu')

        self.max_pooling = layers.MaxPooling2D((2, 2))
        self.flatten = layers.Flatten()
        
        self.linear_1 = layers.Dense(64, activation='relu')
        self.linear_2 = layers.Dense(10)

    def call(self, x):
        x = self.conv_1(x)
        x = self.max_pooling(x)
        x = self.conv_2(x)
        x = self.max_pooling(x)
        x = self.conv_3(x)
        x = self.flatten(x)
        x = self.linear_1(x)
        x = self.linear_2(x)
        return x

model = CNN()

In [17]:
train_x = tf.convert_to_tensor(train_x, dtype=tf.float32)
test_x = tf.convert_to_tensor(test_x, dtype=tf.float32)

train_y = tf.convert_to_tensor(train_y)
test_y = tf.convert_to_tensor(test_y)


train_db = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(100)
test_db = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(100)

In [21]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = optimizers.Adam()

NameError: name 'optimizers' is not defined

In [18]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [19]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        prediction = model(images)
        loss = loss_function(labels, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, prediction)


@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_function(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [20]:
EPOCHS = 10


for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_db:
        train_step(images, labels)

    for test_images, test_labels in test_db:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}%, Test Loss: {}, Test Accuracy: {}%'
        
    print(template.format(epoch + 1,
                          train_loss.result(), 
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

NameError: in user code:

    /var/folders/y5/6xfh1jfd0xs0c2ybtgl5t7gr0000gn/T/ipykernel_39455/1087865665.py:5 train_step  *
        loss = loss_function(labels, prediction)

    NameError: name 'loss_function' is not defined
